This script is a repository for all successful code written and tested in
the wrangling phase.  Goal is to have a single place to run script to get from 
three unique data sets down to two cleaned and concat-able .csv files.

Data is pulled from S3 'worm-begin' bucket (object lock - enabled) and returned to the 'wrangled-1' bucket (object-lock enabled).

These two .csv files will serve as the basis for the next step, Natural Language Processing (NLP) preprcessing.

Loading required libraries and creating functions to enable push/pull from buckets via Boto3 and s3fs.

In [39]:
import pandas as pd
import numpy as np
import s3fs
import os
import io
import boto3

import s3fs
fs = s3fs.S3FileSystem(anon=False,key='###',secret='###')

from dotenv import load_dotenv
load_dotenv(verbose=True)

def aws_session(region_name='us-east-1'):
    return boto3.session.Session(aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'), #looks for any .env file
                                aws_secret_access_key=os.getenv('AWS_ACCESS_KEY_SECRET'), #Has to be in same directory
                                region_name=region_name) #from above

def make_bucket(name, acl): 
    session = aws_session()
    s3_resource = session.resource('s3')
    return s3_resource.create_bucket(Bucket=name, ACL=acl)

def upload_file_to_bucket(bucket_name, file_path):
    session = aws_session()
    s3_resource = session.resource('s3')
    file_dir, file_name = os.path.split(file_path)

    bucket = s3_resource.Bucket(bucket_name)
    bucket.upload_file(
      Filename=file_path,
      Key=file_name,
      ExtraArgs={'ACL': 'public-read'}
    )

    s3_url = f"https://{bucket_name}.s3.amazonaws.com/{file_name}"
    return s3_url

## s3_url = upload_file_to_bucket('worm-begin','lyrics_25k.csv')
## print(s3_url) 
## s3_url = upload_file_to_bucket('worm-begin','album_details_25k.csv')
## print(s3_url)
## s3_url = upload_file_to_bucket('worm-begin','songs_details_25k.csv')
## print(s3_url)

def download_file_from_bucket(bucket_name, s3_key, dst_path):
    session = aws_session()
    s3_resource = session.resource('s3')
    bucket = s3_resource.Bucket(bucket_name)
    bucket.download_file(Key=s3_key, Filename=dst_path)

The genres data set comes in two .csv files.  Downloading both using boto3 which requires an existing file on the local directory. 

In [40]:
!touch genres_lyrics.csv
!touch genres_genres.csv

Download both halves of the genres dataset (found from Kaggle at:https://www.kaggle.com/neisse/scrapped-lyrics-from-6-genres?select=lyrics-data.csv).

In [41]:
download_file_from_bucket('worm-begin','genres_lyrics_data.csv','genres_lyrics.csv')
with open('genres_lyrics.csv') as fo:
    lyrics_df = pd.read_csv(fo)

download_file_from_bucket('worm-begin','genres_artists_data.csv','genres_genres.csv')
with open('genres_genres.csv') as fo:
    genres_df = pd.read_csv(fo)

In [42]:
lyrics_df.describe(include='all')

,ALink,SName,SLink,Lyric,Idiom
count,209522,209522,209522,209484,206375
unique,2993,128083,167499,164789,47
top,/chris-brown/,Intro,/edu-gueda/beijo-bom-part-loubet.html,Instrumental,ENGLISH
freq,1176,80,6,592,114723


In [43]:
lyrics_df.head()

,ALink,SName,SLink,Lyric,Idiom
0,/10000-maniacs/,More Than This,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...,ENGLISH
1,/10000-maniacs/,Because The Night,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close...",ENGLISH
2,/10000-maniacs/,These Are Days,/10000-maniacs/these-are-days.html,These are. These are days you'll remember. Nev...,ENGLISH
3,/10000-maniacs/,A Campfire Song,/10000-maniacs/a-campfire-song.html,"A lie to say, ""O my mountain has coal veins an...",ENGLISH
4,/10000-maniacs/,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,Trudging slowly over wet sand. Back to the ben...,ENGLISH


In [44]:
genres_df.describe(include='all')

,Artist,Songs,Popularity,Link,Genre,Genres
count,3242,3242.000000,3242.000000,3242,3242,3238
unique,2940,NaN,NaN,2940,6,1777
top,Fotos,NaN,NaN,/rick-rock/,Rock,Sertanejo
freq,3,NaN,NaN,3,797,295
mean,NaN,53.907465,1.522455,NaN,NaN,NaN
std,NaN,78.212829,6.965121,NaN,NaN,NaN
min,NaN,0.000000,0.000000,NaN,NaN,NaN
25%,NaN,4.000000,0.000000,NaN,NaN,NaN
50%,NaN,17.000000,0.000000,NaN,NaN,NaN
75%,NaN,77.000000,0.500000,NaN,NaN,NaN


In [45]:
genres_df.head()

,Artist,Songs,Popularity,Link,Genre,Genres
0,10000 Maniacs,110,0.3,/10000-maniacs/,Rock,Rock; Pop; Electronica; Dance; J-Pop/J-Rock; G...
1,12 Stones,75,0.3,/12-stones/,Rock,Rock; Gospel/Religioso; Hard Rock; Grunge; Roc...
2,311,196,0.5,/311/,Rock,Rock; Surf Music; Reggae; Ska; Pop/Rock; Rock ...
3,4 Non Blondes,15,7.5,/4-non-blondes/,Rock,Rock; Pop/Rock; Rock Alternativo; Grunge; Blue...
4,A Cruz Está Vazia,13,0.0,/a-cruz-esta-vazia/,Rock,Rock


Reduce genres_df to just artist-name (the key with lyrics_df) and drop dupes.

In [46]:
genres1_df = pd.DataFrame(genres_df, columns=['Link','Genre'])
genres2_df = genres1_df.rename(columns={'Link':'artist_name','Genre':'genre'})
genres2_df[genres2_df.duplicated(keep = False)]

,artist_name,genre


In [47]:
genres2_df.describe(include='all')

,artist_name,genre
count,3242,3242
unique,2940,6
top,/rick-rock/,Rock
freq,3,797


In [48]:
genres2_df.isnull().sum()

artist_name    0
genre          0
dtype: int64

In [49]:
genres2_df[genres2_df.duplicated(subset=['artist_name'])]

,artist_name,genre
1947,/10000-maniacs/,Pop
1975,/beastie-boys/,Rock
2002,/claudio-henrique-seven-music/,Sertanejo
2008,/crowded-house/,Rock
2012,/danni-carlos/,Rock
...,...,...
3227,/tati-quebra-barraco/,Hip Hop
3228,/valesca-popozuda/,Pop
3229,/vine-rodry/,Pop
3234,/leandro-sapucahy/,Pop


In [50]:
genres2_df.drop_duplicates(subset=['artist_name'],inplace=True)

In [51]:
genres2_df.describe(include='all')

,artist_name,genre
count,2940,2940
unique,2940,6
top,/ze-maholics/,Rock
freq,1,755


In [52]:
genres2_df[genres2_df.duplicated(subset=['artist_name'])]

,artist_name,genre


Reorder lyrics_df columns and rename IAW naming convention. Drop duplicates.
Drop all but the ENGLISH lyrics.

In [53]:
lyrics2_df = lyrics_df.rename(columns={'ALink':'artist_name','SName':'song_name','SLink':'link','Lyric':'lyrics','Idiom':'language'})
lyrics3_df = (lyrics2_df[lyrics2_df['language']=='ENGLISH'])
lyrics3_df[lyrics3_df.duplicated(keep = False)]
%time

CPU times: user 1 µs, sys: 7 µs, total: 8 µs
Wall time: 10 µs


In [54]:
lyrics3_df.describe(include='all')

,artist_name,song_name,link,lyrics,language
count,114723,114723,114723,114723,114723
unique,1198,70261,91611,90796,1
top,/chris-brown/,Home,/m-i-a/bring-the-noize.html,[This song is an instrumental.].,ENGLISH
freq,1174,61,6,17,114723


In [55]:
lyrics3_df.isnull().sum()

artist_name    0
song_name      0
link           0
lyrics         0
language       0
dtype: int64

Checking for and then removing duplicate lyrics (multiple artists doing the same song).  Almost 24,000.

In [56]:
lyrics3_df[lyrics3_df.duplicated(subset=['lyrics'])]

,artist_name,song_name,link,lyrics,language
23,/10000-maniacs/,A Campfire Song,/10000-maniacs/a-campfire-song.html,"A lie to say, ""O my mountain has coal veins an...",ENGLISH
24,/10000-maniacs/,A Room For Everything,/10000-maniacs/a-room-for-everything.html,"You were looking away from me, western skies c...",ENGLISH
25,/10000-maniacs/,Across The Fields,/10000-maniacs/across-the-fields.html,"Well they left then in the morning, a hundred ...",ENGLISH
26,/10000-maniacs/,All That Never Happens,/10000-maniacs/all-that-never-happens.html,"She walks alone on the brick lane,. the breeze...",ENGLISH
29,/10000-maniacs/,Anthem For Doomed Youth,/10000-maniacs/anthem-for-doomed-youth.html,For whom do the bells toll. When sentenced to ...,ENGLISH
...,...,...,...,...,...
207587,/sambo/,I Feel Good,/sambo/i-feel-good.html,"Whooooau!. I feel good, I knew that I would no...",ENGLISH
207592,/sambo/,Mercedes Benz,/sambo/mercedes-benz.html,"Oh Lord, won't you buy me a Mercedes Benz ?. M...",ENGLISH
207614,/sambo/,Suddenly I See,/sambo/suddenly-i-see.html,"Her face is a map of the world, is a map of th...",ENGLISH
207615,/sambo/,"Sunday, Bloody Sunday",/sambo/sunday-bloody-sunday.html,I can't believe the news today. I can't close ...,ENGLISH


In [57]:
lyrics3_df.drop_duplicates(subset=['lyrics'],inplace=True)

<ipython-input-57-c957743e557b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lyrics3_df.drop_duplicates(subset=['lyrics'],inplace=True)


In [58]:
lyrics3_df[lyrics3_df.duplicated(subset=['lyrics'])]

,artist_name,song_name,link,lyrics,language


Merge lyrics_df with genre_df to add genre to a single df with the lyrics.

In [59]:
merged_genre_df = pd.merge(lyrics3_df,genres2_df,on = 'artist_name') 
%time

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


In [60]:
merged_genre_df.describe(include='all')

,artist_name,song_name,link,lyrics,language,genre
count,86391,86391,86391,86391,86391,86391
unique,1147,66856,86391,86391,1,6
top,/elvis-presley/,Intro,/paul-mccartney/ebony-and-ivory.html,Beauty and grace is what touches me most. Good...,ENGLISH,Rock
freq,747,50,1,1,86391,47408


Strip the ' / ' from 'singer_name', so we can pd.merge later with decades dataset. 

In [61]:
merged_genre_df['artist_name'].replace('(/)','',regex=True, inplace = True)

In [62]:
merged_genre_df.head()

,artist_name,song_name,link,lyrics,language,genre
0,10000-maniacs,More Than This,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...,ENGLISH,Rock
1,10000-maniacs,Because The Night,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close...",ENGLISH,Rock
2,10000-maniacs,These Are Days,/10000-maniacs/these-are-days.html,These are. These are days you'll remember. Nev...,ENGLISH,Rock
3,10000-maniacs,A Campfire Song,/10000-maniacs/a-campfire-song.html,"A lie to say, ""O my mountain has coal veins an...",ENGLISH,Rock
4,10000-maniacs,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,Trudging slowly over wet sand. Back to the ben...,ENGLISH,Rock


Adding a column for the orginal.csv name, in case we merge datasets later.

In [63]:
merged2_genre_df = pd.DataFrame((merged_genre_df), columns = ['original_csv','artist_name','song_name','link','lyrics','language','genre','date'])
merged2_genre_df['original_csv'] = 'genres_csv'

In [64]:
merged2_genre_df.head()

,original_csv,artist_name,song_name,link,lyrics,language,genre,date
0,genres_csv,10000-maniacs,More Than This,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...,ENGLISH,Rock,NaN
1,genres_csv,10000-maniacs,Because The Night,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close...",ENGLISH,Rock,NaN
2,genres_csv,10000-maniacs,These Are Days,/10000-maniacs/these-are-days.html,These are. These are days you'll remember. Nev...,ENGLISH,Rock,NaN
3,genres_csv,10000-maniacs,A Campfire Song,/10000-maniacs/a-campfire-song.html,"A lie to say, ""O my mountain has coal veins an...",ENGLISH,Rock,NaN
4,genres_csv,10000-maniacs,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,Trudging slowly over wet sand. Back to the ben...,ENGLISH,Rock,NaN


Write the dataframe to a csv.  When you use df.to_csv in jupyter notebook you need to give it a path, a landing spot...
First round I did it in a notebook, with no path.  Crashed my browser.

In [65]:
!touch merged5_genre_df.csv

In [66]:
merged2_genre_df.to_csv('merged5_genre_df.csv', index=False)

Push to S3 bucket.

In [67]:
upload_file_to_bucket('wrangled-1','merged5_genre_df.csv')
%time

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 16.7 µs


Converting decades_tcc .csv file to data frame standard.

This dataset came from: https://www.kaggle.com/saurabhshahane/music-dataset-1950-to-2019

I'm using the s3fs method to pull from the bucket this time, vice the boto3 method used above.

In [68]:
decades_df = pd.read_csv('s3://worm-begin/decades_tcc_ceds_music.csv')
%time

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 2.62 µs


In [69]:
decades_df.head()

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,...,0.380299,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.0
1,4,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,...,0.001284,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.0
2,6,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,...,0.002770,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.0
3,10,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,...,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.0
4,12,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,...,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.0


In [70]:
decades_df.describe(include='all')

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
count,28372.000000,28372,28372,28372.000000,28372,28372,28372.000000,28372.000000,28372.000000,28372.000000,...,28372.000000,28372.000000,28372.000000,28372.000000,2.837200e+04,28372.000000,28372.000000,28372.000000,28372,28372.000000
unique,NaN,5426,23689,NaN,7,28372,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN
top,NaN,johnny cash,tonight,NaN,pop,latest sink fast race nearly strongest trials ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sadness,NaN
freq,NaN,190,17,NaN,7042,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6096,NaN
mean,42946.323558,NaN,NaN,1990.236888,NaN,NaN,73.028444,0.021112,0.118396,0.120973,...,0.129389,0.030996,0.533348,0.665249,3.392347e-01,0.080049,0.532864,0.569875,NaN,0.425187
std,24749.325492,NaN,NaN,18.487463,NaN,NaN,41.829831,0.052370,0.178684,0.172200,...,0.181143,0.071652,0.173218,0.108434,3.267143e-01,0.211245,0.250972,0.244385,NaN,0.264107
min,0.000000,NaN,NaN,1950.000000,NaN,NaN,1.000000,0.000291,0.000284,0.000291,...,0.000284,0.000289,0.005415,0.000000,2.811248e-07,0.000000,0.000000,0.000000,NaN,0.014286
25%,20391.250000,NaN,NaN,1975.000000,NaN,NaN,42.000000,0.000923,0.001120,0.001170,...,0.001144,0.000993,0.412975,0.595364,3.423598e-02,0.000000,0.329143,0.380361,NaN,0.185714
50%,45405.500000,NaN,NaN,1991.000000,NaN,NaN,63.000000,0.001462,0.002506,0.006579,...,0.005263,0.001754,0.538612,0.679050,2.259028e-01,0.000085,0.539365,0.580567,NaN,0.414286
75%,64090.500000,NaN,NaN,2007.000000,NaN,NaN,93.000000,0.004049,0.192608,0.197793,...,0.235113,0.032622,0.656666,0.749026,6.325298e-01,0.009335,0.738252,0.772766,NaN,0.642857


In [71]:
decades_df.isnull().sum()

Unnamed: 0                  0
artist_name                 0
track_name                  0
release_date                0
genre                       0
lyrics                      0
len                         0
dating                      0
violence                    0
world/life                  0
night/time                  0
shake the audience          0
family/gospel               0
romantic                    0
communication               0
obscene                     0
music                       0
movement/places             0
light/visual perceptions    0
family/spiritual            0
like/girls                  0
sadness                     0
feelings                    0
danceability                0
loudness                    0
acousticness                0
instrumentalness            0
valence                     0
energy                      0
topic                       0
age                         0
dtype: int64

In [72]:
decades_df[decades_df.duplicated(subset=['lyrics'])]

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age


Convert to the format genres data set is in.

In [73]:
decades2_df = pd.DataFrame((decades_df), columns=['original_csv','artist_name','track_name','link','lyrics','language','genre','release_date'])
decades2_df['original_csv'] = 'decades_tcc'
decades3_df = decades2_df.rename(columns={'track_name':'song_name','release_date':'date'})

In [74]:
decades3_df.head()

,original_csv,artist_name,song_name,link,lyrics,language,genre,date
0,decades_tcc,mukesh,mohabbat bhi jhoothi,NaN,hold time feel break feel untrue convince spea...,NaN,pop,1950
1,decades_tcc,frankie laine,i believe,NaN,believe drop rain fall grow believe darkest ni...,NaN,pop,1950
2,decades_tcc,johnnie ray,cry,NaN,sweetheart send letter goodbye secret feel bet...,NaN,pop,1950
3,decades_tcc,pérez prado,patricia,NaN,kiss lips want stroll charm mambo chacha merin...,NaN,pop,1950
4,decades_tcc,giorgos papadopoulos,apopse eida oneiro,NaN,till darling till matter know till dream live ...,NaN,pop,1950


In [77]:
decades3_df.date.unique().sum()

138915

In [78]:
decades3_df[decades3_df.duplicated(keep = False)]

,original_csv,artist_name,song_name,link,lyrics,language,genre,date


In [79]:
decades3_df.date.unique().sum()

138915

In [ ]:
!touch decades4_df.csv
decades3_df.to_csv('decades4_df.csv', index= False)
upload_file_to_bucket('wrangled-1','decades4_df.csv')
%time